<a href="https://colab.research.google.com/github/abSETEd/check_phone/blob/main/googlemapsver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Primeiro teste**

In [ ]:
!pip install requests pandas bs4 openpyxl
!pip install odfpy
!pip install pyexcel_ods3
!pip install googlemaps
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
from google.colab import files  # Para o upload no Colab
import io  # Para processar o arquivo enviado
import re  # Para expressões regulares
import os  # Adicionado pra usar os.path
import pyexcel_ods3  # Pra salvar ODS

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(arquivo_carregado))
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='openpyxl')
        elif nome_arquivo.endswith('.ods'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='odf')  # Usa odfpy
        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        if 'Site' not in df.columns:
            raise ValueError("A planilha precisa ter uma coluna chamada 'Site'.")

        # Formata a coluna 'Site' adicionando 'https://' se necessário
        df['Site'] = df['Site'].apply(lambda x: f"https://{x}" if not str(x).startswith(('http://', 'https://')) else x)

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar o site e extrair telefones
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
        telefones = re.findall(padrao_telefone, texto)
        telefones_unicos = list(set(telefones))

        return telefones_unicos if telefones_unicos else []
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return []

# Função para consultar o google maps via busca no google
def consultar_google_maps(url):
  try:
    dominio = url.replace('httpm://','').replace('http://','').split('/')[0]
    query = f"site:maps.google.com {dominio}"
    busca_url = f"https://www.google.com/search?q={query.replace(' ', '+')}"

    headers = {'User-Agent': 'Mozila/5.0'}
    resposta = requests.get(busca_url, headers=headers, timeout=10)
    resposta.raise_for_status()

    soup = BeautifulSoup(resposta.text, 'html.perser')
    textp = soup.get_text()

    padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
    telefones = re.findall(padrao_telefone, Texto)
    telefones_unicos = list(set(telefones))

    return telefones_unicos if telefones_unicos else[]
  except requests.exceptions.RequestException as e :
    print(f"Erro ao buscar {url} no Google Maps: {e}")
    return[]

'''
        if telefones_unicos:
            return ', '.join(telefones_unicos)
        else:
            return "Nenhum telefone encontrado"
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return "Erro na consulta"
'''
# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    uploaded = files.upload()

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha: {nome_arquivo}")

    resultados = []
    for index, row in df_urls.iterrows():
        url = row['Site']
        print(f"Consultando: {url}")

        resultado = consultar_site(url)
        resultados.append({'Site': url, 'Telefone': resultado})
        print(f"Telefones para {url}: {resultado}")

        time.sleep(2)

    # Salva no mesmo formato do arquivo de entrada
    nome_base = os.path.splitext(nome_arquivo)[0]
    extensao = os.path.splitext(nome_arquivo)[1]
    nome_saida = f"{nome_base}_resultados{extensao}"

    df_resultados = pd.DataFrame(resultados)

    if extensao == '.csv':
        df_resultados.to_csv(nome_saida, index=False)
    elif extensao == '.xlsx':
        df_resultados.to_excel(nome_saida, index=False, engine='openpyxl')
    elif extensao == '.ods':
        # Usa pyexcel_ods3 pra salvar ODS
        pyexcel_ods3.save_data(nome_saida, {'Sheet1': [df_resultados.columns.tolist()] + df_resultados.values.tolist()})

    print(f"Resultados salvos em '{nome_saida}'. Você pode baixar o arquivo na aba 'Files'!")

# Executa o programa
if __name__ == "__main__":
    main()

Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):


In [ ]:
!pip install requests pandas bs4 openpyxl
!pip install odfpy
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
import pyexcel_ods3  # Para ODS
from google.colab import files  # Para o upload no Colab
import re  # Para expressões regulares

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(arquivo_carregado)
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(arquivo_carregado, engine='openpyxl')
        elif nome_arquivo.endswith('.odf'):
            df = pd.read_execel(arquivo_carregado, engine='.odf')

        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        if 'Site' not in df.columns:
            raise ValueError("A planilha precisa ter uma coluna chamada 'Site'.")

        # Formata a coluna 'Site' adicionando 'https://' se necessário
        df['Site'] = df['Site'].apply(lambda x: f"https://{x}" if not str(x).startswith(('http://', 'https://')) else x)

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar o site e extrair telefones
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
        telefones = re.findall(padrao_telefone, texto)
        telefones_unicos = list(set(telefones))

        if telefones_unicos:
            return ', '.join(telefones_unicos)
        else:
            return "Nenhum telefone encontrado"
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return "Erro na consulta"

# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    # Interface de upload no Colab
    uploaded = files.upload()

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    # Pega o nome e conteúdo do arquivo enviado
    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    # Carrega a planilha
    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha: {nome_arquivo}")

    # Lista para armazenar resultados
    resultados = []

    # Itera pelos sites
    for index, row in df_urls.iterrows():
        url = row['Site']
        print(f"Consultando: {url}")

        resultado = consultar_site(url)
        resultados.append({'Site': url, 'Telefone': resultado})
        print(f"Telefones para {url}: {resultado}")

        time.sleep(2)  # Pausa pra evitar bloqueios

    # Salva os resultados no mesmo formato do arquivo de entrada
    nome_base = os.path.splitext(nome_arquivo)[0]  # Pega o nome sem extensão
    extensao = os.path.splitext(nome_arquivo)[1]  # Pega a extensão
    nome_saida = f"{nome_base}_resultados{extensao}"

    df_resultados = pd.DataFrame(resultados)

    if extensao == '.csv':
        df_resultados.to_csv(nome_saida, index=False)
    elif extensao == '.xlsx':
        df_resultados.to_excel(nome_saida, index=False, engine='openpyxl')
    elif extensao == '.ods':
        df_resultados.to_excel(nome_saida, index=False, engine='openpyxl')  # Converte pra ODS abaixo
        pd.DataFrame.to_excel = lambda self, path, *args, **kwargs: pyexcel_ods3.save_data(path, {'Sheet1': [self.columns.tolist()] + self.values.tolist()})
        df_resultados.to_excel(nome_saida)

    print(f"Resultados salvos em '{nome_saida}'. Você pode baixar o arquivo na aba 'Files'!")

# Executa o programa
if __name__ == "__main__":
    !pip install pyexcel-ods3  # Instala suporte pra ODS no Colab
    main()